In [1]:
import numpy as np
import pandas as pd

df=pd.read_csv('clean_data.csv')
df.head()

,Text,Emotion,Clean_text
0,i didnt feel humiliated,sadness,didnt feel humiliated
1,i can go from feeling so hopeless to so damned...,sadness,feeling hopeless damned hopeful cares awake
2,im grabbing a minute to post i feel greedy wrong,anger,im grabbing minute post feel greedy wrong
3,i am ever feeling nostalgic about the fireplac...,love,feeling nostalgic fireplace know property
4,i am feeling grouchy,anger,feeling grouchy


In [2]:
df.shape

(42174, 3)

In [3]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM, Conv1D
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout
from tensorflow.keras import regularizers

In [4]:
voc_size=10000
onehot_repr=[one_hot(words,voc_size)for words in df['Clean_text']] 
print(onehot_repr)

[[4762, 8915, 5688], [427, 1469, 1486, 283, 3658, 6556], [6220, 92, 3660, 3285, 8915, 3816, 8051], [427, 301, 8329, 7428, 6923], [427, 2272], [8831, 427, 9050, 9192, 4517, 3991, 9549], [8831, 5836, 7419, 7901, 9045, 8831, 7099, 3259, 7689, 3404, 8915, 9007, 5743], [8915, 7720, 8071, 966, 2515, 5739, 2585, 2219], [8510, 6743, 8915, 8510, 6928, 6412, 6106], [8915, 9556], [8915, 9007, 7721, 7737, 8827], [8915, 5390, 1478, 5890, 1232, 671, 8417, 9709], [6452, 1844, 701, 5739, 8915, 6287], [8915, 7871, 4135, 518], [203, 1834, 5248, 5087, 8305, 4749, 40, 7172, 9982, 701, 6254, 5521, 8071, 8555, 5116, 7768, 8832, 6661, 4503, 9050, 4376], [8915, 385, 6478], [4762, 8915, 1173], [8915, 5825, 6466, 701], [7291, 427, 3166, 666, 7737, 6766, 5218, 8844, 514, 666, 348], [8915, 1162, 4302, 9837, 7553, 8577], [8915, 5466, 8144, 359], [427, 3411, 2735, 9607, 7288, 8915, 820, 4400, 5165, 8638, 3987, 160, 8098, 7813, 3218, 5066, 7467], [427, 1437, 2209], [4903, 7288, 9622, 6165, 6270, 2627, 5871, 427, 782

In [5]:
type(onehot_repr)

list

In [6]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 4762 8915 5688]
 [   0    0    0 ...  283 3658 6556]
 [   0    0    0 ... 8915 3816 8051]
 ...
 [   0    0    0 ... 7911 7012 4970]
 [   0    0    0 ...  427 2735 9499]
 [   0    0    0 ... 4376 8728 5761]]


In [7]:
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Conv1D(100, kernel_size=3, activation = "relu"))
model1.add(Bidirectional(LSTM(150, return_sequences=True)))
model1.add(Dropout(0.5))
model1.add(Bidirectional(LSTM(50)))
model1.add(Dropout(0.2))
model1.add(Dense(50, activation='relu'))
model1.add(Dropout(0.2))
model1.add(Dense(50, activation='relu'))
model1.add(Dropout(0.2))
model1.add(Flatten())
model1.add(Dense(150, kernel_regularizer=regularizers.l2(0.01), activation='relu'))
model1.add(Dropout(0.5))
model1.add(Dense(6,activation='softmax'))
model1.compile(loss='categorical_crossentropy',optimizer=tf.compat.v1.train.AdamOptimizer(
    learning_rate=0.00001, beta1=0.9, beta2=0.999, epsilon=10e-08, use_locking=False,
    name='Adam'),metrics=['accuracy'])
print(model1.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            400000    
_________________________________________________________________
conv1d (Conv1D)              (None, 18, 100)           12100     
_________________________________________________________________
bidirectional (Bidirectional (None, 18, 300)           301200    
_________________________________________________________________
dropout (Dropout)            (None, 18, 300)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 100)               140400    
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 50)                5

In [8]:
dummy=pd.get_dummies(df['Emotion'])
df=pd.concat([df, dummy], axis=1)
df.drop('Emotion', axis=1, inplace=True)
df.head()

,Text,Clean_text,anger,fear,happy,love,sadness,surprise
0,i didnt feel humiliated,didnt feel humiliated,0,0,0,0,1,0
1,i can go from feeling so hopeless to so damned...,feeling hopeless damned hopeful cares awake,0,0,0,0,1,0
2,im grabbing a minute to post i feel greedy wrong,im grabbing minute post feel greedy wrong,1,0,0,0,0,0
3,i am ever feeling nostalgic about the fireplac...,feeling nostalgic fireplace know property,0,0,0,1,0,0
4,i am feeling grouchy,feeling grouchy,1,0,0,0,0,0


In [9]:
y=df[['anger', 'fear', 'happy', 'love', 'sadness', 'surprise']]
len(embedded_docs),y.shape

(42174, (42174, 6))

In [10]:
type(y)

pandas.core.frame.DataFrame

In [11]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [12]:
X_final.shape,y_final.shape

((42174, 20), (42174, 6))

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.25, random_state=42)

In [14]:
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=100, batch_size=64)

Epoch 1/100
495/495 [==============================] - 17s 20ms/step - loss: 2.5187 - accuracy: 0.1664 - val_loss: 2.4879 - val_accuracy: 0.1784
Epoch 2/100
495/495 [==============================] - 9s 19ms/step - loss: 2.4589 - accuracy: 0.1710 - val_loss: 2.4303 - val_accuracy: 0.1843
Epoch 3/100
495/495 [==============================] - 9s 19ms/step - loss: 2.4034 - accuracy: 0.1762 - val_loss: 2.3769 - val_accuracy: 0.1819
Epoch 4/100
495/495 [==============================] - 9s 19ms/step - loss: 2.3518 - accuracy: 0.1762 - val_loss: 2.3271 - val_accuracy: 0.1808
Epoch 5/100
495/495 [==============================] - 9s 19ms/step - loss: 2.3037 - accuracy: 0.1820 - val_loss: 2.2805 - val_accuracy: 0.1874
Epoch 6/100
495/495 [==============================] - 9s 19ms/step - loss: 2.2585 - accuracy: 0.1861 - val_loss: 2.2364 - val_accuracy: 0.1938
Epoch 7/100
495/495 [==============================] - 9s 19ms/step - loss: 2.2155 - accuracy: 0.1951 - val_loss: 2.1932 - val_accuracy

495/495 [==============================] - 9s 19ms/step - loss: 0.5976 - accuracy: 0.8525 - val_loss: 0.8347 - val_accuracy: 0.7824
Epoch 58/100
495/495 [==============================] - 9s 19ms/step - loss: 0.5908 - accuracy: 0.8561 - val_loss: 0.8559 - val_accuracy: 0.7757
Epoch 59/100
495/495 [==============================] - 9s 19ms/step - loss: 0.5852 - accuracy: 0.8590 - val_loss: 0.8439 - val_accuracy: 0.7839
Epoch 60/100
495/495 [==============================] - 9s 19ms/step - loss: 0.5702 - accuracy: 0.8644 - val_loss: 0.8334 - val_accuracy: 0.7883
Epoch 61/100
495/495 [==============================] - 9s 19ms/step - loss: 0.5603 - accuracy: 0.8690 - val_loss: 0.8421 - val_accuracy: 0.7863
Epoch 62/100
495/495 [==============================] - 9s 19ms/step - loss: 0.5575 - accuracy: 0.8709 - val_loss: 0.8350 - val_accuracy: 0.7895
Epoch 63/100
495/495 [==============================] - 9s 19ms/step - loss: 0.5469 - accuracy: 0.8758 - val_loss: 0.8300 - val_accuracy: 0.792